In [6]:
import nbimporter
from Cleaning_Dataset import clean_diamonds, standard

Importing Jupyter notebook from Cleaning_Dataset.ipynb


In [14]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
!brew install libomp
from xgboost import XGBRegressor
from sklearn.model_selection import learning_curve, GridSearchCV

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 3 taps (homebrew/cask-versions, homebrew/core and homebrew/cask).
==> Updated Formulae
Updated 1 formula.
==> Updated Casks
axe-edit-iii                             openbazaar
brave-browser-dev                        pagico
dashlane                                 psychopy
electrum                                 refined-github-safari
epichrome                                rocks-n-diamonds
fsnotes                                  rsyncosx
glyphs                                   sauerbraten
hookshot                                 sessionrestore
jami                                     softorino-youtube-converter
jedit-omega                              syntax-highlight
mic-drop                                 tableplus
multitouch                               trilium-notes
odrive

To reinstall 11.0.0, run `brew reinstall libomp`


In [7]:
original_train = pd.read_csv("../Data/train.csv")

In [8]:
original_test = pd.read_csv("../Data/test.csv")

In [9]:
original_train_cleaned = clean_diamonds(original_train)

In [10]:
original_test_cleaned = clean_diamonds(original_test)

### Installation

In [1]:
!pip3 install xgboost

     |████████████████████████████████| 1.2 MB 3.6 MB/s eta 0:00:01     |███████▊                        | 286 kB 3.6 MB/s eta 0:00:01


In [2]:
!pip3 install --upgrade xgboost

Requirement already up-to-date: xgboost in /Users/Jaime/opt/anaconda3/envs/ironhack/lib/python3.8/site-packages (1.2.1)


I also had to do "brew install libomp" before importing the XGBRegressor

### Building the model

In [11]:
X = original_train_cleaned.drop("price", axis = 1)
y = original_train_cleaned["price"]
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=123)

I´m getting the parameters from https://www.kaggle.com/jayatou/xgbregressor-with-gridsearchcvÇ

In [64]:
model = XGBRegressor()
params = {'nthread':[3, 4, 5], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.018, 0.02, 0.022], #so called `eta` value
              'max_depth': [8],
              'min_child_weight': [1, 1.5, 2],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [1600, 1650, 1700, 1750]}
grid_search = GridSearchCV(model,
                           param_grid=params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 69.4min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 147.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 176.8min finished


[15:15:46] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[15:15:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:16:14] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=...
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
                                    verbosity=None),
             n_

In [52]:
grid_search.best_score_

0.9930487467957066

In [53]:
best_XGB = grid_search.best_estimator_
best_XGB.score(X_test, y_test)

0.9929055060257501

In [54]:
grid_search.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.02,
 'max_depth': 8,
 'min_child_weight': 2,
 'n_estimators': 1650,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.7}

In [55]:
best_XGB.fit(X, y)

[11:40:23] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[11:40:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:41:15] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=8,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=1650, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

### Real Test

In [56]:
prediction = best_XGB.predict(original_test_cleaned)

In [57]:
len(prediction)

13485

In [58]:
XGB_df = pd.DataFrame(prediction, columns = ["price"])

In [59]:
XGB_df["id"] = XGB_df.index

In [60]:
XGB_df = XGB_df[["id", "price"]]

In [61]:
XGB_df.to_csv(r"XGB_df.csv", index = False)

In [62]:
!mv XGB_df.csv ../Predictions/XGB_df.csv